In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/hwyTrafficPred

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/hwyTrafficPred


In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from toolkits.datasets import load_next_5min
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os

In [3]:
class CNNRegression(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cnnLayer = nn.Sequential(
            nn.Conv2d(in_channels=5, out_channels=16, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(1, 1, 0),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(1, 1, 0),
        )

        self.fcLayer = nn.Sequential(
            nn.Linear(32 * 1 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
            nn.Sigmoid(),
        )

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.to(self.device)

    def forward(self, x) -> torch.Tensor:
        x = self.cnnLayer(x)
        x = x.flatten(1)
        x = self.fcLayer(x)
        return x

    def save(self, ckpt_dir: str) -> None:
        torch.save(self.state_dict(), os.path.join(ckpt_dir, 'model.pth'))

    def load(self, ckpt_dir: str) -> None:
        self.load_state_dict(torch.load(os.path.join(ckpt_dir, 'model.pth')))

In [4]:
# Hyperparams for training
batch_size = 1024
lr = 1e-3
n_epochs = 20

# Prepare datasets and dataloaders
trainDataset, testDataset = load_next_5min(ckpt_dir='./toolkits/cnndataset')
trainLoader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
testLoader = DataLoader(testDataset, batch_size=batch_size, shuffle=False)

model = CNNRegression()
optimizer = optim.Adam(params=model.parameters(), lr=lr, weight_decay=1e-8)

In [5]:
for epoch in range(n_epochs):
    # ========= Training =========
    model.train()

    # Record Info in training
    train_loss = []

    for batch in tqdm(trainLoader):
        X, y = batch
        logits = model(X.to(model.device))
        loss = F.mse_loss(logits, y.to(model.device))

        # Compute gradients and update model params
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())

    # Compute the average train_loss
    avg_loss = sum(train_loss) / len(train_loss)
    print(f"[ Train | {epoch + 1:d}/{n_epochs:d} ] loss = {avg_loss:.5f}")

    # ========= Testing =========
    model.eval()

    # Record Info in testing
    test_loss = []

    for batch in tqdm(testLoader):
        X, y = batch
        logits = model(X.to(model.device))
        loss = F.mse_loss(logits, y.to(model.device))

        # Compute gradients and update model params
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        test_loss.append(loss.item())

    # Compute the average train_loss
    avg_loss = sum(test_loss) / len(test_loss)
    print(f"[ Test | {epoch + 1:d}/{n_epochs:d} ] loss = {avg_loss:.5f}")


# Save model after training
model.save('./model/v2/')

  0%|          | 0/7060 [00:00<?, ?it/s]

[ Train | 1/20 ] loss = 0.00475


  0%|          | 0/1765 [00:00<?, ?it/s]

[ Test | 1/20 ] loss = 0.00425


  0%|          | 0/7060 [00:00<?, ?it/s]

...

[ Test | 19/20 ] loss = 0.00381


  0%|          | 0/7060 [00:00<?, ?it/s]

[ Train | 20/20 ] loss = 0.00386


  0%|          | 0/1765 [00:00<?, ?it/s]

[ Test | 20/20 ] loss = 0.00380
